In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel(io='data/total_df2.xlsx', engine = 'openpyxl')
df.head()

index                title  year  audience                 date  \
0      1               범죄도시 2  2022  12693023  2022-05-18 00:00:00   
1      2              탑건: 매버릭  2022   8171923  2022-06-22 00:00:00   
2      3            한산: 용의 출현  2022   7262885  2022-07-27 00:00:00   
3      4           공조2: 인터내셔날  2022   6948819  2022-09-07 00:00:00   
4      5  닥터 스트레인지: 대혼돈의 멀티버스  2022   5884600  2022-05-04 00:00:00   

                                                 url        genre  \
0  https://movie.naver.com/movie/bi/mi/basic.nave...       범죄, 액션   
1  https://movie.naver.com/movie/bi/mi/basic.nave...           액션   
2  https://movie.naver.com/movie/bi/mi/basic.nave...      액션, 드라마   
3  https://movie.naver.com/movie/bi/mi/basic.nave...      액션, 코미디   
4  https://movie.naver.com/movie/bi/mi/basic.nave...  액션, 판타지, 모험   

                                               actor country  \
0        마동석(마석도), 손석구(강해상), 최귀화(전일만), 마동석, 손석구, 최귀화    대한민국   
1  톰 크루즈(매버릭), 마일즈 텔러(루스터), 제니퍼 코넬리(페니), 톰 크루즈, 마...      미국   
2  박해일(이순신), 변요한(와키자카 야스하루), 안성기(어영담), 박해일, 변요한, 안성기    대한민국   
3            현빈(림철령), 유해진(강진태), 윤아(박민영), 현빈, 유해진, 윤아    대한민국   
4  베네딕트 컴버배치(닥터 스트레인지), 엘리자베스 올슨(완다 막시모프 / 스칼렛 위치...      미국   

           director      age  net_score  \
0          이상용, 이상용  15세 관람가       9.32   
1  조셉 코신스키, 조셉 코신스키  12세 관람가       9.76   
2          김한민, 김한민  12세 관람가       8.75   
3          이석훈, 이석훈  15세 관람가       8.44   
4      샘 레이미, 샘 레이미  12세 관람가       7.22   

                                            synopsis  
0  ['가리봉동 소탕작전 후 4년 뒤,금천서 강력반은 베트남으로 도주한 용의자를 인도받...  
1  ['최고의 파일럿이자 전설적인 인물 매버릭(톰 크루즈)은 자신이 졸업한 훈련학교 교...  
2  ['1592년 4월, 조선은 임진왜란 발발 후 단 15일 만에 왜군에 한양을 빼앗기...  
3  ['공조 이즈 백! 이번엔 삼각 공조다!남한으로 숨어든 글로벌 범죄 조직을 잡기 위...  
4  ['끝없이 균열되는 차원과 뒤엉킨 시공간의 멀티버스가 열리며오랜 동료들, 그리고 차...

In [3]:
from konlpy.tag import Komoran
tagger = Komoran()
from collections import Counter

[0.019s][warning][os,thread] Attempt to protect stack guard pages failed (0x000000016d528000-0x000000016d534000).
[0.019s][warning][os,thread] Attempt to deallocate stack guard pages failed.


In [ ]:
df.synopsis

In [ ]:
df[df.year==2022]['synopsis']

In [16]:
synopsis = df['synopsis'].values.tolist()

In [21]:
tagged = [tagger.pos(str(sent)) for sent in synopsis]

In [ ]:
tagged

In [90]:
nng_list = []
for sent in tagged:
    sent_nng = [word for word, tag in sent if tag == 'NNG' and len(word) > 1 and word not in ['시작', '사람', '자신', '한편']]
    nng_list.append(sent_nng)

In [ ]:
nng_list

In [92]:
df['synopsis_nng'] = nng_list

In [93]:
import itertools

In [94]:
all_nng = list(itertools.chain.from_iterable(nng_list))

In [95]:
count_nng = Counter(all_nng)

In [ ]:
count_nng.most_common()[0:30]

In [ ]:
unique_words = list(count_nng)
unique_words

In [ ]:
word_index = {word: i for i, word in enumerate(unique_words)}
word_index

In [99]:
occurs = np.zeros([len(tagged), len(unique_words)])
np.shape(occurs)

(2150, 8410)

In [100]:
for i, sent in enumerate(tagged):
    for word, tag in sent:
        if tag == 'NNG' and len(word) > 1 and word not in ['시작', '사람', '자신', '한편']:
            index = word_index[word] 
            occurs[i][index] = 1

In [101]:
co_occurs = occurs.T.dot(occurs)

In [ ]:
co_occurs_top = []
for i in range(len(unique_words)):
    for j in range(len(unique_words)):
        if i < j and co_occurs[i][j] >= 50:
            co_occurs_top.append((i,j))
co_occurs_top

In [103]:
len(co_occurs_top)

73

In [104]:
for i, j in co_occurs_top:
    print(unique_words[i], ',',unique_words[j])

마지막 , 사랑
위기 , 사건
위기 , 도움
위기 , 친구
위기 , 사랑
운명 , 사랑
조직 , 사건
수사 , 사건
마음 , 사건
마음 , 남자
마음 , 친구
마음 , 사랑
마음 , 여자
서로 , 사랑
추적 , 사건
사건 , 도움
사건 , 납치
사건 , 현장
사건 , 해결
사건 , 사실
사건 , 경찰
사건 , 친구
사건 , 범인
사건 , 발생
사건 , 발견
사건 , 사랑
사건 , 여자
사건 , 살해
남자 , 사랑
남자 , 여자
인간 , 세계
처음 , 사랑
도움 , 사랑
성공 , 사랑
세상 , 친구
세상 , 사랑
순간 , 사랑
아내 , 남편
아내 , 사랑
죽음 , 친구
죽음 , 사랑
결심 , 사랑
시간 , 사랑
사실 , 친구
사실 , 발견
사실 , 사랑
사실 , 여자
인생 , 사랑
생각 , 사랑
세계 , 사랑
친구 , 발견
친구 , 생활
친구 , 사랑
친구 , 아버지
친구 , 사이
친구 , 여자
남편 , 사랑
아들 , 사랑
아들 , 아버지
관계 , 사랑
발견 , 사랑
아이 , 사랑
생활 , 사랑
생활 , 결혼
모습 , 사랑
가족 , 사랑
행복 , 사랑
사랑 , 아버지
사랑 , 여인
사랑 , 사이
사랑 , 여자
사랑 , 결혼
사랑 , 감정


In [ ]:
co_occurs_set = set()
for (i, j) in co_occurs_top:
    co_occurs_set.add(i)
    co_occurs_set.add(j)
co_occurs_set


In [106]:
len(co_occurs_set)

48

In [ ]:
for i in co_occurs_set:
    print(unique_words[i])

In [108]:
co_occurs_top_words = [unique_words[i] for i in co_occurs_set]

In [ ]:
final_words = []
for word, count in count_nng.most_common():
    if count < 150 or len(final_words) >= 30:
        break
    if word in co_occurs_top_words:
        final_words.append(word)
final_words

In [ ]:
df.synopsis_nng

In [ ]:
'''
data = {
    "사랑" : [1, 2, 4, ... , 2000],
    "죽음" :  [1, 2, 4, ... , 2000],
    ...,
    "아버지": [1, 2, 4, ... , 2000],
}
'''

In [126]:
data = {}
for i in final_words:
    idx_list = []
    for idx, row in df.iterrows():
        for nng in row.synopsis_nng:
            if i == nng:
                idx_list.append(idx)
    count_idx = list(Counter(idx_list).most_common())
    data[i] = count_idx

In [ ]:
data

In [128]:
import json

In [129]:
with open("data/cooccur_idx.json", 'w', encoding="UTF-8") as file:
    json.dump(data, file, indent=4, ensure_ascii=False)